In [ ]:
from utils import *
from metadata import *
import utils
utils.init_dev(utils.get_dev())
utils.allow_growth()

%load_ext autoreload
# %reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
 


In [6]:
tree=ori_tree
tree2=tree=slim_tree(tree,'imagenet1k')

In [7]:
def find_child(tree_,node):
    res=[]
    try:
        for node in nx.dfs_preorder_nodes(tree_,node):
            imagepath=get_imagepath(node).strip('.tar')
            if osp.exists(imagepath) and tree_.node[node]['nchild'] == 0:
                res.append(node)
    except Exception as inst : 
        print inst,'wrong'        
    return res 

In [ ]:
@chdir_to_root
def nimg_per_class(tree_):
    all_nimg = []
    for node in nx.dfs_preorder_nodes(tree_, 'fall11'):
        imagepath = get_imagepath(node).strip('.tar')
        if osp.exists(imagepath) and tree_.node[node]['nchild'] == 0:
            nimg = len(os.listdir(imagepath))
            all_nimg.append(nimg)
            if nimg == 0:
                print node
                rm(imagepath)
        elif tree_.node[node]['nchild'] >0 and node in imagenet1k:
            nimg_=0
            for node_ in find_child(tree_,node):
                imagepath=get_imagepath(node_).strip('.tar') 
                nimg_+=len(os.listdir(imagepath)) 
#             if nimg_>2000:
#                 print node
            all_nimg.append(nimg_) 
                
    return np.sort(all_nimg)[::-1]


In [ ]:
def stat_info(imagenet1k):
    nimgs=[]
    for node in imagenet1k:
        imagepath = get_imagepath(node).strip('.tar')
        if osp.exists(imagepath):
            nimgs.append(len(os.listdir(imagepath))) 
        else:
            nimg=0
            for node_ in find_child(ori_tree,node): 
                imagepath=get_imagepath(node_).strip('.tar')
                nimg+=len(os.listdir(imagepath)) 
            if nimg ==0:
                print node_
            else:
                nimgs.append(nimg) 

    nimgs = np.sort(nimgs)[::-1]
    sum(nimgs),sum(nimgs)/len(nimgs),len(nimgs)

    _=plt.hist(nimgs)
    plt.figure()
    plt.plot(nimgs[:])
    plt.yscale('log') 
    return nimgs

In [ ]:
for imagenet in [imagenet1k,imagenet7k,imagenet22k]:
    stat_info(imagenet)

In [29]:
nimgs={}
os.chdir('/mnt/nfs1703/kchen/imagenet-raw/')
for d in os.listdir('.'):
    nimgs[d]=len(os.listdir(d))

names,vals = np.array(nimgs.keys()),np.array(nimgs.values())

len(nimgs)

nimgs=utils.unpickle('./nimgs.pkl')